In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [2]:
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

In [7]:
import cPickle as pickle

In [8]:
with open('./flavor_compund_map.pkl', 'r') as f:
    raw_flav_ing = pickle.load(f)

In [9]:
ingredient_bow = ' '.join(raw_flav_ing.values())

## Seperating individual ingredients

I need to-
take out common connective words
lower
remove punct?

In [10]:
ing_words = word_tokenize(ingredient_bow)

## ing_words

In [11]:
with open('./data/recipes.pkl', 'r') as f:
    recipe_raw = pickle.load(f)

In [12]:
recipe_comb = {}
for f in recipe_raw:
    recipe_comb.update(f)

In [13]:
raw_flav_ing

In [14]:
cocktail_ing_lists = recipe_comb.values()
cocktail_ing_bow = [ing[0].lower() for ing in cocktail_ing_lists]
len(set(cocktail_ing_bow))

In [15]:
ingredient_bow[:1000]

In [16]:
wcs = Counter(ingredient_bow.split(','))
wc_common = wcs.most_common()

In [17]:
ingredients_list_from_fen = [ing[0].strip().replace(',','') for ing in wc_common if ing[1]>1]
fen_set = set(ingredients_list_from_fen)

In [18]:
len(fen_set)

In [20]:
fen_set.intersection(set(cocktail_ing_bow))

In [29]:
def replace_with_ing_lists(ing_st):
    new_list = []
    potential_ing = ing_st.split(',')
    for ing in potential_ing:
        if ing.strip().replace(',','') in fen_set:
            new_list.append(ing.strip().replace(',',''))
    return new_list

In [30]:
with open('./flavor_compund_map.pkl', 'r') as f:
    raw_flav_ing = pickle.load(f)

In [31]:
for comp, ing_st in raw_flav_ing.iteritems():
    raw_flav_ing[comp] = replace_with_ing_lists(ing_st)

In [32]:
flav_ing_dict = {k:v for k,v in raw_flav_ing.iteritems() if v}

Splitting on the commas, while easy, misses ingredients that appear as below

In [28]:
raw_flav_ing['2,6-DIMETHYL-4-HEPTANONE']

In [33]:
ing_flav_dict = defaultdict(list)

for k, v in flav_ing_dict.iteritems():
    for ing in v:
        ing_flav_dict[ing].append(k)

In [35]:
lengths = []
for k, v in ing_flav_dict.iteritems():
    lengths.append((k, len(v)))

In [64]:
ing_counts = pd.DataFrame(lengths).sort_values(1, ascending =False)
ing_counts.head(500)

In [62]:
ing_counts.index = ing_counts[0]

In [37]:
_ = ing_counts[1].hist(bins = 15, log =True)

In [178]:
with open('./data/first_ing_flav_dict.pkl', 'w') as f:
    pickle.dump(ing_flav_dict, f)